In [ ]:
# Q1-Q2
R.<x>  = PolynomialRing(GF(2))
FF.<w> = GF(2^6,x^6 + x^4 + x^3 + x +1)
n = 2^6-1

m1 = w.minimal_polynomial()
m3 = (w^3).minimal_polynomial()
m5 = (w^5).minimal_polynomial()
m7 = (w^7).minimal_polynomial()

g = lcm(lcm(lcm(m1,m3),m5),m7)
print(g)

C = codes.CyclicCode(g,n)
print(C.minimum_distance())

# print(C.encoders_available())
# print(C.bch_bound())

# str_msg = "000001111100000111110000011111000001111"
# k = len(str_msg)
# msg = [int(str_msg[i]) for i in range(k)]
# msgvec = vector(GF(2), msg)
# E = codes.encoders.CyclicCodeVectorEncoder(C)
# cword = E.encode(msgvec)
# print(cword)
# rpoly = sum([x^(i)*cword[i] for i in range(63)])
# print(rpoly)

# E1 = codes.encoders.CyclicCodePolynomialEncoder(C)
# msgpoly = sum([x^(i)*msg[i] for i in range(k)])
# rpoly1 = E.encode(msgpoly)
# print(rpoly1)

# msg2 = vector(GF(2), (1,1,1,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,))
# cword2 = E.encode(msg2)
# # print(cword2)

# n_err = 4

# Chan  = channels.StaticErrorRateChannel(GF(2)^n, n_err)
# rword = Chan.transmit(cword2)
# # print(rword)

# str_rword = "010000000000011111000001011011111010101011000001111011010110111"
# rword = [int(str_rword[i]) for i in range(n)]
# D = codes.decoders.CyclicCodeSurroundingBCHDecoder(C)
# D.decode_to_code(rword)


x^24 + x^23 + x^21 + x^19 + x^18 + x^16 + x^15 + x^14 + x^12 + x^11 + x^10 + x^8 + x^7 + x^5 + 1


In [71]:
# Peterson-Gorenstein-Zierler Decoding
q = 2^6
t = 4
d = 2*t + 1
FF.<w> = FiniteField(q)
R.<x>  = GF(2)[]
RR.<z> = FF[]

m1 = w.minimal_polynomial()
m3 = (w^3).minimal_polynomial()
m5 = (w^5).minimal_polynomial()
m7 = (w^7).minimal_polynomial()

g = lcm(lcm(lcm(m1,m3),m5),m7)
print(g)

C = codes.CyclicCode(g,n)

n = q -1
k = n - g.degree()

C = codes.CyclicCode(g, n)
# print(C.bch_bound())

msg = random_vector(GF(2), k)
print(msg)

E = codes.encoders.CyclicCodeVectorEncoder(C)
cword = E.encode(msg)

codes.encoders.CyclicCodePolynomialEncoder

n_err = t
Chan = channels.StaticErrorRateChannel(GF(2)^n, n_err)
rword = Chan(cword)
print(rword - cword)


# r_wrd = '111000011111111001011101111010011011001101100000000110101001001'
# rli = [int(r_wrd[i]) for i in range(63)]
# rli = (0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1)

# rword = vector(GF(2), rli)
print(rword)

# D = codes.decoders.CyclicCodeSurroundingBCHDecoder(C);
# print(D.decode_to_code(rword))
# msg = D.decode_to_message(rword)


# Step 1
rpoly = sum([x^i*rword[i] for i in range(n)])
print(rpoly)
S = [rpoly(w^j) for j in range(1,2*t+1)]
print(rword in C)

# Step 2
r = t
arrS = [ S[j:j+r] for j in range(r) ]
matS = Matrix(arrS)
print(matS.dimensions())
print(matS.determinant())

while matS.determinant() == 0 and r > 0:
    r -= 1
    arrS = [ S[j+r-1:j:-1] for j in range(r) ]
    matS = Matrix(arrS)
print(matS.dimensions())
print(matS.determinant())
    
vec  = vector([-S[j] for j in range(r,2*r)]) 
coeff_sigma = matS.solve_right(vec)
print(coeff_sigma)

sigma_poly = 1+sum([coeff_sigma[i]*x^(r-i) for i in range(len(coeff_sigma))])
print(sigma_poly)
# factor(sigma_poly)

print(S)

# Step 2 - BM algorithm
u = 0; L = 0; sig = [x^0]; b =[x^0];
while u < 2*t:
    sigpoly = sig[u]
    delta_u = sum([S[u-i]*sigpoly[i] for i in range(L+1)] )
    if delta_u == 0:
        sig.append(sig[u])
        b.append(x*b[u])
    else:
        sig.append(sig[u]-delta_u*x*b[u])
        if 2*L > u:
            b.append(x*b[u])
        else:
            b.append(sig[u]/delta_u)
            L = u + 1 - L
#     print(sig, b)
    u = u + 1    
print("BM alg. Success:", sig[2*t] == sigma_poly)    

# Step 2: Sugiyama alg

# f = x^(2*t)
# s = sum([S[i]*x^i for i in range(2*t)])

# r = [f, s]; b = [0,1]
# while r[-1].degree() >= t:
# #     print(r[-1],"\n",r[-2])
#     rtmp = r[-2] % r[-1]
#     h = (r[-2] - rtmp) / r[-1]
#     r.append(rtmp)
#     b.append(b[-2]-h*b[-1])
    
# print("XGCD alg. factor:", b[-1]/ sigma)    

# Step 3
err_pos = []
for j in range(n):
    if sigma_poly(w^j) == 0:
        err_pos.append(n-j)
#         print(n-j)
print(err_pos)
# # Step 4

# coeffMatE = Matrix([[(w^j)^err_pos[u] for u in range(r-1,-1,-1) ] for j in range(r)])
# print(coeffMatE.determinant())
# vec = vector(S[0:r])
# errs = coeffMatE.solve_right(vec)
# print(errs)

x^24 + x^23 + x^21 + x^19 + x^18 + x^16 + x^15 + x^14 + x^12 + x^11 + x^10 + x^8 + x^7 + x^5 + 1
(1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0)
(1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1)
x^62 + x^61 + x^58 + x^55 + x^53 + x^49 + x^48 + x^45 + x^44 + x^42 + x^41 + x^40 + x^39 + x^35 + x^34 + x^30 + x^27 + x^26 + x^25 + x^22 + x^18 + x^17 + x^16 + x^15 + x^13 + x^11 + x^7 + x^6 + x^5 + x^4 + x^2 + x + 1
False
(4, 4)
w
(4, 4)
w
(w^4 + w, w^3 + 1, w^5 + w^3 + w^2 + 1, w^4 + w^3 + 1)
(w^4 + w)*x^4 + (w^3 + 1)*x^3 + (w^5 + w^3 + w^2 + 1)*x^2 + (w^4 + w^3 + 1)*x + 1
[w^4 + w^3 + 1, w^5 + w^3 

In [132]:
#Q5
m = 6
R.<z> = GF(2)[]
FF.<w> = GF(2^6, modulus=z^6 + z^4 + z^3 + z + 1)

ZZ = IntegerRing()
def binrepr(a): # binary representation of elements in GF(2^m)
    a_int = ZZ(a.integer_representation())
    seq = a_int.digits(2, padto=m)
    return [seq[i] for i in range(m-1,-1,-1)]
print(binrepr(w^1))
print(binrepr(w^45))

RR.<x> = FF[]
g = x^5 + (w^4+1)*x + (w^4+1)
t = g.degree()

for a in FF:
    d, u, v = xgcd(x-a,g)
    item = (x^(2^m)-x)*u % g
    print(a, ":", [item[i] for i in range(t+1)])

[0, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 1, 1]
0 : [w^3 + w^2 + w + 1, w^3 + w + 1, w^4 + w^3 + w^2, w^4 + w^2 + w, 0, 0]
w : [w^4 + w^3 + w^2 + 1, w^4 + w^2, w^5 + w, w^4 + w^3 + w^2 + w + 1, w^5 + w^3 + 1, 0]
w^2 : [w^5 + w^2, w^5 + w^4 + w^3, w^4 + w^3 + w, w^5 + w^3 + w^2, w^5 + w + 1, 0]
w^3 : [w^2 + w + 1, w^2 + w, w^4 + w^3 + w^2, 0, w^4 + w^2, 0]
w^4 : [w^5 + 1, w^2 + 1, w^5 + w^2 + w, w^4 + w^3 + w^2 + w, w^5 + w^3 + w^2 + 1, 0]
w^5 : [1, w^4 + w, w^4 + w^3 + w, w^5 + w^3, w^5 + w^4 + w^3 + w^2 + w + 1, 0]
w^4 + w^3 + w + 1 : [w^5 + w^4 + w^3 + w^2 + 1, w^5 + w^2 + w + 1, w^2 + 1, w^3 + w^2 + 1, 1, 0]
w^5 + w^4 + w^2 + w : [w^2 + w + 1, w^4 + w^3 + w^2 + w + 1, w^5 + w^4 + w^3 + w^2, w^5 + w^4 + w^3 + w + 1, w^5 + w^3 + w^2, 0]
w^5 + w^4 + w^2 + w + 1 : [w^5 + w^4 + w + 1, w^4 + 1, w^4 + w^2 + w + 1, w^5 + w + 1, w, 0]
w^5 + w^4 + w^2 + 1 : [w^5 + w^3 + w^2 + w, w^5, w^5 + w^2, w^4 + w, w + 1, 0]
w^5 + w^4 + 1 : [w^4 + w^2 + w, w^4 + w^3, w^4 + w^3 + w + 1, w^5 + w^4 + 1, w^5 + w^4 + w